In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # This is the analysis pipeline for the calculation of tau and branching factor of neural data from the hippocampus. 
# 
# The data are take from: https://datadryad.org/stash/dataset/doi:10.7272/Q61N7ZC3

# ### Importing the branching factor/tau estimator
# 
# For documentation regards the estimator see https://mrestimator.readthedocs.io/en/latest/doc/gettingstarted.html, Spitzner, F. P., Dehning, J., Wilting, J., Hagemann, A., P. Neto, J., Zierenberg, J., & Priesemann, V. (2021).

# ### Importing relevant non-standard modules
# All of which can be found on github https://github.com/JanBellingrath/Hippocampal_Neuron_Data_Processing


import pipeline.initiation_functions as init
from collections import namedtuple


# ### Defining each animal via its short name and its directory
Animal = namedtuple('Animal', {'short_name', 'directory'})


conley = Animal('/home/bellijjy/Conley.tar/Conley', 'con')
dave = Animal('/home/bellijjy/Dave.tar/Dave/Dave', 'dav')
chapati = Animal('/home/bellijjy/Chapati.tar/Chapati/Chapati', 'cha')
corriander = Animal('/home/bellijjy/Corriander.tar/Corriander', 'Cor')
dudley = Animal('/home/bellijjy/Dudley/dud', 'dud')
bond = Animal('/home/bellijjy/Bond/bon', 'bon')
frank = Animal('/local2/Jan/Frank/Frank', 'fra')
government = Animal('/local2/Jan/Government/Government/gov', 'gov')
egypt = Animal('/local2/Jan/Egypt/Egypt/egy', 'egy')
remy = Animal('/local2/Jan/Remy/Remy/remy', 'remy')
five = Animal("/home/dekorvyb/Downloads/Fiv", "Fiv")
bon = Animal("/home/dekorvyb/Downloads/Bon", "bon")



#egypt = Animal('/local2/Jan/Egypt/Egypt', 'egy')
#remy = Animal('/local2/Jan/Remy/Remy', 'remy')




animals_dict = {#'con': Animal('con','/home/bellijjy/Conley.tar/Conley'),
           #'Cor': Animal('Cor','/home/bellijjy/Corriander.tar/Corriander'),
           #'cha': Animal('cha','/home/bellijjy/Chapati.tar/Chapati/Chapati'),
           #'dav': Animal('dav','/home/bellijjy/Dave.tar/Dave/Dave'),
           #'dud': Animal('dud','/home/bellijjy/Dudley'),
           'fra' : Animal('fra', '/local2/Jan/Frank/Frank'),
           'gov' : Animal('gov', '/local2/Jan/Government/Government'),
           'egy' : Animal('egy', '/local2/Jan/Egypt/Egypt'), 
           'remy': Animal('remy', '/local2/Jan/Remy/Remy'),
           "Fiv" : Animal("Fiv", "/home/dekorvyb/Downloads/Fiv"),
           "bon" : Animal("bon", "/home/dekorvyb/Downloads/Bon")}



# animals to analyze
animal_list = [ "bon"] #'egy', 'con', 'dav', 'fra', 'gov', 'remy', 'bon', 'dud', 'Cor', "Fiv"

# areas to analyze
area_list = ['CA1'] #CA3
len_time_chunk = 90

init.intrinsic_time_scale_estimation_for_all_animals(animal_list,
                                                     area_list,
                                                     len_time_chunk,
                                                     animals_dict,
                                                     mode = "debug")



#dav ca3 done
#cor ca1 done
#rest not yet



INFO     Loaded mrestimator v0.1.8, writing to /tmp/mre_dekorvyb/
INFO     Using numba for parallelizable functions


Processing for animal: bon, area: CA1
animal_full_names: [Animal(short_name='/local2/Jan/Egypt/Egypt', directory='egy'), Animal(short_name='/local2/Jan/Remy/Remy', directory='remy'), Animal(short_name='/local2/Jan/Government/Government', directory='gov'), Animal(short_name='/local2/Jan/Frank/Frank', directory='fra'), Animal(short_name='/home/dekorvyb/Downloads/Bon', directory='bon')]
task_files: ['/local2/Jan/Egypt/Egypt/egytask02.mat', '/local2/Jan/Egypt/Egypt/egytask06.mat', '/local2/Jan/Egypt/Egypt/egytask07.mat', '/local2/Jan/Egypt/Egypt/egytask04.mat', '/local2/Jan/Egypt/Egypt/egytask05.mat', '/local2/Jan/Egypt/Egypt/egytask11.mat', '/local2/Jan/Egypt/Egypt/egytask09.mat', '/local2/Jan/Egypt/Egypt/egytask08.mat', '/local2/Jan/Egypt/Egypt/egytask10.mat', '/local2/Jan/Egypt/Egypt/egytask01.mat', '/local2/Jan/Egypt/Egypt/egytask03.mat']
task_files: ['/local2/Jan/Remy/Remy/remytask36.mat', '/local2/Jan/Remy/Remy/remytask37.mat', '/local2/Jan/Remy/Remy/remytask35.mat']
task_files: ['/l